# Initialization

In [22]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
import pandas as pd
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

# Basic Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
flattened_lattice = avail_lattice_base.flatten()

# Env Lattices

In [4]:
Private_sector_Owned = (pickle.load( open( "Privately Owned Housing.p", "rb" ) ))
Social_Sector_Rental = (pickle.load( open( "Social Sector Rental Housing.p", "rb" ) ))
Free_Sector_rental = (pickle.load( open( "Free Sector Rental Housing.p", "rb" ) )) 
Restaurants_and_Cafes = (pickle.load( open( "Restaurants and Cafe.p", "rb" ) )) 
Offices = (pickle.load( open( "Offices.p", "rb" ) )) 
Retail = (pickle.load( open( "Retail Stores.p", "rb" ) )) 
Parking = (pickle.load( open( "Parking.p", "rb" ) )) 

# Stencils

In [5]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [6]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [7]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [8]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [9]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

In [10]:
stencil_full_floor = tg.create_stencil("moore",int(10/2) )*0
stencil_full_floor[:,:,int(10/2)] = 1
stencil_full_floor.set_index([0,0,0], 0)
#print(stencil_full_floor)

# Agent_sizes

In [11]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
P_H = 415 # Private Housing
S_H= 301 # Social Housing
F_H=403 # Free sector Housing
R_C= 19 # Restaurant and Cafe 
R= 73 # Retail stores
O=93 # Offices
P=51 # Parking


# All possible origin points 

In [12]:
available_index_list= list(np.argwhere(flattened_lattice==1).flatten())

# ABM Simulation

In [13]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
 
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(selected_neigh_1D_id)
        #print(free_neighs_full_lattice)

            

In [14]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [15]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [16]:
def calculate_cost_for_positions(Location_list,Enviornment_lattice,number_of_cells_to_be_occupied):
    All_viz_options = []
    for indexing, item in enumerate(Location_list):
        #print(indexing)
        Agent = agent(np.unravel_index(item,init_avail_lattice.shape),stencil_cuboid,9)

        occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

        env = {"availibility": occ_lattice_sim,"enviornment": Enviornment_lattice}


        env_1 = environment(env,Agent,number_of_cells_to_be_occupied,"one_neighbour_occupy_cubish_behaviour")

        env_availability_viz = []
        env_list =[env_1]
        number_steps = max(map(lambda e:e.number_of_iterations,env_list))    
        for a in range(number_steps):
            # print(env.availibility)
            #print(env.agent_origin)

            for e in env_list:
                if a < e.number_of_iterations :
                    #print(a)
                    #print(e.number_of_iterations)
                    if e.method_name == "one_neighbour_occupy_squareness_moore":
                        e.one_neighbour_occupy_squareness_moore()

                    elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                        e.one_neighbour_occupy_cubish_agents()

                    elif e.method_name == "random_occupy_squareness_agents" :
                        e.random_occupy_squareness_agents()

                    elif e.method_name == "random_occupy_cubish_agents" :
                        e.random_occupy_cubish_agents()  

                    elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                        e.random_occupy_cubish_von_neumann_agents()                           

                    elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                        e.one_neighbour_occupy_squareness_von_neumann()                


                    elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                        e.one_neighbour_occupy_squareness_behaviour()  

                    elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                        e.one_neighbour_occupy_cubish_behaviour()  

                    elif e.method_name == "argmax_occupy_von_neumann" :
                        e.argmax_occupy_von_neumann()  

        env_availability_viz.append(e.availibility)

        All_viz_options.append(env_availability_viz)

    all_viz_array= np.array(All_viz_options)
    all_viz_array_shape = all_viz_array.shape
    reshaped_all_viz= all_viz_array.reshape(all_viz_array_shape[0]*all_viz_array_shape[1],all_viz_array_shape[2],all_viz_array_shape[3],all_viz_array_shape[4])
    sum_of_values = []
    for lattice in reshaped_all_viz:
        indexes= np.argwhere(lattice.flatten()==9).flatten()
        values= env["enviornment"].flatten()[indexes]
        total = int(np.sum(values))
        sum_of_values.append(total)
    return sum_of_values

In [18]:
Cost_for_PH = np.array(calculate_cost_for_positions(available_index_list,Private_sector_Owned,P_H)) # Privately owned
Cost_for_SR_H = np.array(calculate_cost_for_positions(available_index_list,Social_Sector_Rental,S_H)) # Social rental
Cost_for_FS_Rental = np.array(calculate_cost_for_positions(available_index_list,Free_Sector_rental,F_H)) # Free-sector Rental
Cost_for_R = np.array(calculate_cost_for_positions(available_index_list,Restaurants_and_Cafes,R_C))# Restaurants
Cost_for_O = np.array(calculate_cost_for_positions(available_index_list,Offices,R)) # Offices
Cost_for_RE = np.array(calculate_cost_for_positions(available_index_list,Retail,O)) # Retail
Cost_for_PA = np.array(calculate_cost_for_positions(available_index_list,Parking,P)) # Parking


In [19]:
pickle.dump( Cost_for_PH, open( "Cost_for_PH.p", "wb" ) )
pickle.dump( Cost_for_SR_H, open( "Cost_for_SR_H.p", "wb" ) )
pickle.dump( Cost_for_FS_Rental, open( "Cost_for_FS_Rental.p", "wb" ) )
pickle.dump( Cost_for_R, open( "Cost_for_R.p", "wb" ) )
pickle.dump( Cost_for_O, open( "Cost_for_O.p", "wb" ) )
pickle.dump( Cost_for_RE, open( "Cost_for_RE.p", "wb" ) )
pickle.dump( Cost_for_PA, open( "Cost_for_PA.p", "wb" ) )


In [20]:
sorted_PH= np.sort(Cost_for_PH)
sorted_SR_H= np.sort(Cost_for_SR_H)
sorted_FS_Rental= np.sort(Cost_for_FS_Rental)
sorted_R= np.sort(Cost_for_R)
sorted_O= np.sort(Cost_for_O)
sorted_RE= np.sort(Cost_for_RE)
sorted_PA= np.sort(Cost_for_PA)

In [37]:
all_values =np.array( [sorted_PH] + [sorted_SR_H] + [sorted_FS_Rental] + [sorted_R] + [sorted_O] + [sorted_RE] + [sorted_PA])

In [38]:
df = pd.DataFrame(all_values.T, columns =["Private_Housing", "Social_Rental_Housing", "Free_Sector_Housing", "Restaurant", "Offices", "Retail", "Parking"])

In [39]:
df

,Private_Housing,Social_Rental_Housing,Free_Sector_Housing,Restaurant,Offices,Retail,Parking
0,2228,1600,2161,97,354,484,104
1,2229,1600,2162,98,354,484,104
2,2229,1600,2162,98,354,492,104
3,2233,1602,2163,98,355,492,110
4,2233,1602,2163,98,356,497,110
...,...,...,...,...,...,...,...
1354,2450,1942,2604,136,499,682,412
1355,2450,1942,2604,136,499,682,412
1356,2450,1942,2604,136,499,682,412
1357,2450,1942,2604,136,499,682,412


In [49]:
df.to_excel("Evaluation_framework.xlsx")

PermissionError: [Errno 13] Permission denied: 'Evaluation_framework.xlsx'

In [73]:
sorted_PH[-1]

2450

In [67]:
sorted_SR_H[0]

1600

In [68]:
sorted_FS_Rental[0]

2161

In [69]:
sorted_R[0]

97

In [70]:
sorted_O[0]

354

In [71]:
sorted_RE[0]

484

In [72]:
sorted_PA[0]

104

In [88]:
PH_max= np.sum(list(reversed(np.sort(Private_sector_Owned.flatten())))[0:P_H])

In [89]:
SR_max= np.sum(list(reversed(np.sort(Social_Sector_Rental.flatten())))[0:S_H])

In [90]:
FS_max= np.sum(list(reversed(np.sort(Free_Sector_rental.flatten())))[0:F_H])

In [99]:
R_max= np.sum(list(reversed(np.sort(Restaurants_and_Cafes.flatten())))[0:R_C])

In [92]:
O_max= np.sum(list(reversed(np.sort(Offices.flatten())))[0:R])

In [93]:
RE_max= np.sum(list(reversed(np.sort(Retail.flatten())))[0:O])

In [94]:
PA_max= np.sum(list(reversed(np.sort(Parking.flatten())))[0:P])

In [102]:
RE_max

709.6095288149467

In [103]:
PA_max

453.33333333333337